In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Python code in one [module](https://docs.python.org/3/glossary.html#term-module) gains access to the code in another module by the process of [importing](https://docs.python.org/3/glossary.html#term-importing) it. The [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement is the most common way of invoking the import machinery, but it is not the only way. Functions such as [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) and built-in [`__import__()`](https://docs.python.org/3/library/functions.html#import__) can also be used to invoke the import machinery.

The [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement combines two operations; it searches for the named module, then it binds the results of that search to a name in the local scope. The search operation of the `import` statement is defined as a call to the [`__import__()`](https://docs.python.org/3/library/functions.html#import__) function, with the appropriate arguments. The return value of [`__import__()`](https://docs.python.org/3/library/functions.html#import__) is used to perform the name binding operation of the `import` statement. See the `import` statement for the exact details of that name binding operation.

A direct call to [`__import__()`](https://docs.python.org/3/library/functions.html#import__) performs only the module search and, if found, the module creation operation. While certain side-effects may occur, such as the importing of parent packages, and the updating of various caches (including [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules)), only the [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement performs a name binding operation.

When an [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement is executed, the standard builtin [`__import__()`](https://docs.python.org/3/library/functions.html#import__) function is called. Other mechanisms for invoking the import system (such as [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module)) may choose to bypass [`__import__()`](https://docs.python.org/3/library/functions.html#import__) and use their own solutions to implement import semantics.

When a module is first imported, Python searches for the module and if found, it creates a module object [1](https://docs.python.org/3/reference/import.html#fnmo), initializing it. If the named module cannot be found, a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) is raised. Python implements various strategies to search for the named module when the import machinery is invoked. These strategies can be modified and extended by using various hooks described in the sections below.

*Changed in version 3.3:* The import system has been updated to fully implement the second phase of [**PEP 302**](https://www.python.org/dev/peps/pep-0302). There is no longer any implicit import machinery - the full import system is exposed through [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path). In addition, native namespace package support has been implemented (see [**PEP 420**](https://www.python.org/dev/peps/pep-0420)).

> 一个 [模块](https://docs.python.org/3/glossary.html#term-module) 中的 Python 代码通过 [importing](https://docs.python.org/3/glossary.html#term-importing) 过程获得对另一个模块中的代码的访问。[`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句是调用导入机制的最常见方式，但它不是唯一的方式。诸如 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) 和内置的 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 等函数也可以用来调用导入机制。
>
> [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句结合了两个操作；它搜索指定的模块，然后将搜索结果绑定到局部作用域的一个名字。`import`语句的搜索操作被定义为对 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 函数的调用，并带有适当的参数。[`__import__()`](https://docs.python.org/3/library/functions.html#import__) 的返回值被用来执行 `import` 语句的名称绑定操作。关于该名称绑定操作的具体细节，请参见`import`语句。
>
> 直接调用 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 只执行模块搜索，如果找到，则执行模块创建操作。虽然可能会出现某些副作用，例如导入父包，以及更新各种缓存（包括 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules)），但只有 [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句会执行名称绑定操作。
>
> 当[`import`](https://docs.python.org/3/reference/simple_stmts.html#import)语句被执行时，标准的内置 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 函数被调用。其他调用导入系统的机制 (例如 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module)) 可以选择绕过 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 并使用自己的方案来实现导入语义。
>
> 当一个模块第一次被导入时，Python 会搜索这个模块，如果找到了，它会创建一个模块对象 [^1]，并初始化它。如果找不到指定的模块，就会产生 [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) 。当调用导入机制时，Python 实现了各种策略来搜索指定的模块。这些策略可以通过使用下面几节中描述的各种钩子进行修改和扩展。
>
> *3.3 版中的变化：* 导入系统已经更新，以完全实现 [**PEP 302**](https://www.python.org/dev/peps/pep-0302) 的第二阶段。不再有任何隐含的导入机制--完整的导入系统通过[`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path)公开。此外，本地名称空间包支持已经实现（见[**PEP 420**](https://www.python.org/dev/peps/pep-0420)）。

## 5.1. [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib)

The [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) module provides a rich API for interacting with the import system. For example [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) provides a recommended, simpler API than built-in [`__import__()`](https://docs.python.org/3/library/functions.html#import__) for invoking the import machinery. Refer to the [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) library documentation for additional detail.

> [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) 模块提供了丰富的API来与导入系统进行交互。例如 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) 提供了一个推荐的、比内置的 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 更简单的 API 来调用导入机制。请参考 [`importlib`](https://docs.python.org/3/library/importlib.html#module-importlib) 库文档以了解更多细节。

## 5.2. Packages

Python has only one type of module object, and all modules are of this type, regardless of whether the module is implemented in Python, C, or something else. To help organize modules and provide a naming hierarchy, Python has a concept of [packages](https://docs.python.org/3/glossary.html#term-package).

You can think of packages as the directories on a file system and modules as files within directories, but don’t take this analogy too literally since packages and modules need not originate from the file system. For the purposes of this documentation, we’ll use this convenient analogy of directories and files. Like file system directories, packages are organized hierarchically, and packages may themselves contain subpackages, as well as regular modules.

It’s important to keep in mind that all packages are modules, but not all modules are packages. Or put another way, packages are just a special kind of module. Specifically, any module that contains a `__path__` attribute is considered a package.

All modules have a name. Subpackage names are separated from their parent package name by a dot, akin to Python’s standard attribute access syntax. Thus you might have a package called [`email`](https://docs.python.org/3/library/email.html#module-email), which in turn has a subpackage called [`email.mime`](https://docs.python.org/3/library/email.mime.html#module-email.mime) and a module within that subpackage called `email.mime.text`.

> Python 只有一种类型的模块对象，所有的模块都是这种类型的，不管这个模块是用 Python、C 还是其它语言实现的。为了帮助组织模块并提供一个命名层次，Python 有一个 [包（packages）](https://docs.python.org/3/glossary.html#term-package) 的概念。
>
> 你可以把包想象成文件系统中的目录，把模块想象成目录中的文件，但不要把这个比喻看得太重，因为包和模块不一定来自文件系统。在本文档中，我们将使用这个方便的目录和文件的类比。就像文件系统的目录一样，包是分层次组织的，包本身可能包含子包，以及常规模块。
>
> 重要的是要记住，所有的包都是模块，但不是所有的模块都是包。或者换句话说，包只是模块的一种特殊类型。具体来说，任何包含 `__path__` 属性的模块都被认为是一个包。
>
> 所有模块都有一个名字。子包名与它们的父包名用一个点隔开，类似于 Python 的标准属性访问语法。因此你可能有一个名为 [`email`](https://docs.python.org/3/library/email.html#module-email) 的包，它又有一个名为 [`email.mime`](https://docs.python.org/3/library/email.mime.html#module-email.mime) 的子包，以及该子包中的一个模块，名为 `email.mime.text`。

### 5.2.1. Regular packages

Python defines two types of packages, [regular packages](https://docs.python.org/3/glossary.html#term-regular-package) and [namespace packages](https://docs.python.org/3/glossary.html#term-namespace-package). Regular packages are traditional packages as they existed in Python 3.2 and earlier. A regular package is typically implemented as a directory containing an `__init__.py` file. When a regular package is imported, this `__init__.py` file is implicitly executed, and the objects it defines are bound to names in the package’s namespace. The `__init__.py` file can contain the same Python code that any other module can contain, and Python will add some additional attributes to the module when it is imported.

For example, the following file system layout defines a top level `parent` package with three subpackages:

> Python 定义了两种类型的包，[常规包（regular packages）](https://docs.python.org/3/glossary.html#term-regular-package) 和 [名称空间包（namespace packages）](https://docs.python.org/3/glossary.html#term-namespace-package) 。常规包是传统的包，因为它们存在于 Python 3.2 和更早的版本中。一个常规包通常被实现为一个包含 `__init__.py` 文件的目录。当一个常规包被导入时，这个 `__init__.py` 文件被隐式执行，它所定义的对象被绑定到包的名称空间中。`__init__.py`文件可以包含和其他模块一样的 Python 代码，当模块被导入时，Python 会给它添加一些额外的属性。
>
> 例如，下面的文件系统布局定义了一个带有三个子包的顶层 `parent` 包：

Importing `parent.one` will implicitly execute `parent/__init__.py` and `parent/one/__init__.py`. Subsequent imports of `parent.two` or `parent.three` will execute `parent/two/__init__.py` and `parent/three/__init__.py` respectively.

> 导入 `parent.one` 将隐含地执行 `parent/__init__.py` 和 `parent/one/__init__.py`。随后导入 `parent.two` 或 `parent.three` 将分别执行 `parent/two/__init__.py` 和 `parent/three/__init__.py`。

### 5.2.2. Namespace packages

A namespace package is a composite of various [portions](https://docs.python.org/3/glossary.html#term-portion), where each portion contributes a subpackage to the parent package. Portions may reside in different locations on the file system. Portions may also be found in zip files, on the network, or anywhere else that Python searches during import. Namespace packages may or may not correspond directly to objects on the file system; they may be virtual modules that have no concrete representation.

Namespace packages do not use an ordinary list for their `__path__` attribute. They instead use a custom iterable type which will automatically perform a new search for package portions on the next import attempt within that package if the path of their parent package (or [`sys.path`](https://docs.python.org/3/library/sys.html#sys.path) for a top level package) changes.

With namespace packages, there is no `parent/__init__.py` file. In fact, there may be multiple `parent` directories found during import search, where each one is provided by a different portion. Thus `parent/one` may not be physically located next to `parent/two`. In this case, Python will create a namespace package for the top-level `parent` package whenever it or one of its subpackages is imported.

See also [**PEP 420**](https://www.python.org/dev/peps/pep-0420) for the namespace package specification.

> 一个名称空间包是各种[部分（portions）](https://docs.python.org/3/glossary.html#term-portion)的复合体，其中每个部分为父包贡献一个子包。部分可能驻留在文件系统的不同位置。部分也可以在 zip 文件中找到，在网络上，或者在导入过程中被 Python 搜索到的其他地方。名称空间包可能是也可能不是直接对应于文件系统上的对象；它们可能是没有具体表示的虚拟模块。
>
> 名称空间包不使用普通的列表作为它们的 `__path__` 属性。它们使用一个自定义的可迭代类型，如果它们的父包的路径（或者对于顶级包的[`sys.path`](https://docs.python.org/3/library/sys.html#sys.path)）发生变化，在该包内的下一次导入尝试中，将自动对包的部分进行重新搜索。
>
> 对于名称空间包，没有 `parent/__init__.py` 文件。事实上，在导入搜索过程中可能会发现多个 `parent` 目录，其中每一个都是由不同的部分提供。因此，`parent/one `在物理上可能不在 `parent/two` 旁边。在这种情况下，Python 将为顶级的 `parent` 包创建一个名称空间包，只要它或它的一个子包被导入。
>
> 参见 [**PEP 420**](https://www.python.org/dev/peps/pep-0420) 关于命名空间包的规范。

## 5.3. Searching

To begin the search, Python needs the [fully qualified](https://docs.python.org/3/glossary.html#term-qualified-name) name of the module (or package, but for the purposes of this discussion, the difference is immaterial) being imported. This name may come from various arguments to the [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) statement, or from the parameters to the [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) or [`__import__()`](https://docs.python.org/3/library/functions.html#import__) functions.

This name will be used in various phases of the import search, and it may be the dotted path to a submodule, e.g. `foo.bar.baz`. In this case, Python first tries to import `foo`, then `foo.bar`, and finally `foo.bar.baz`. If any of the intermediate imports fail, a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) is raised.

> 为了开始搜索，Python 需要被导入的模块 (或包) 的 [完全保留的（fully qualified）](https://docs.python.org/3/glossary.html#term-qualified-name) 名称，（但在本讨论中，两者的区别并不重要。）这个名字可能来自 [`import`](https://docs.python.org/3/reference/simple_stmts.html#import) 语句的各种参数，或者来自 [`importlib.import_module()`](https://docs.python.org/3/library/importlib.html#importlib.import_module) 或 [`__import__()`](https://docs.python.org/3/library/functions.html#import__) 函数的参数。
>
> 这个名字将在导入搜索的各个阶段使用，它可能是一个子模块的点状路径，例如：`foo.bar.baz`。在这种情况下，Python 首先尝试导入 `foo`，然后是 `foo.bar`，最后是 `foo.bar. baz`。如果任何一个中间导入失败，就会出现 [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) 。

### 5.3.1. The module cache

The first place checked during import search is [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules). This mapping serves as a cache of all modules that have been previously imported, including the intermediate paths. So if `foo.bar.baz` was previously imported, [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) will contain entries for `foo`, `foo.bar`, and `foo.bar.baz`. Each key will have as its value the corresponding module object.

During import, the module name is looked up in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) and if present, the associated value is the module satisfying the import, and the process completes. However, if the value is `None`, then a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) is raised. If the module name is missing, Python will continue searching for the module.

[`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) is writable. Deleting a key may not destroy the associated module (as other modules may hold references to it), but it will invalidate the cache entry for the named module, causing Python to search anew for the named module upon its next import. The key can also be assigned to `None`, forcing the next import of the module to result in a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError).

Beware though, as if you keep a reference to the module object, invalidate its cache entry in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), and then re-import the named module, the two module objects will *not* be the same. By contrast, [`importlib.reload()`](https://docs.python.org/3/library/importlib.html#importlib.reload) will reuse the *same* module object, and simply reinitialise the module contents by rerunning the module’s code.

> 在导入搜索过程中，首先检查的地方是 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules)。这个映射是对所有之前被导入的模块的一个缓存，包括中间路径。因此，如果以前导入过 `foo.bar.baz`，[`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 将包含`foo`，`foo.bar`，和`foo.bar.baz` 的条目。每个键都有相应的模块对象作为其值。
>
> 在导入过程中，模块名称在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中被查找，如果存在，相关的值就是满足导入的模块，这个过程就完成了。然而，如果这个值是 `None`，那么就会产生 [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) 。如果模块名称丢失，Python 将继续搜索模块。
>
> [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules)是可写的。删除一个键可能不会破坏相关的模块 (因为其他模块可能持有对它的引用)，但它将使命名的模块的缓存条目失效，导致 Python 在下次导入时重新搜索命名的模块。这个键也可以分配给 `None` ，迫使下次导入模块时出现 [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError)。
>
> 但要注意，如果你保留对模块对象的引用，使其在  [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 的缓存条目无效，然后重新导入指定的模块，两个模块对象将*不一样*。相比之下，[`importlib.reload()`](https://docs.python.org/3/library/importlib.html#importlib.reload) 将重新使用*相同*的模块对象，并通过重新运行模块的代码简单地重新初始化模块内容。

### 5.3.2. Finders and loaders

If the named module is not found in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), then Python’s import protocol is invoked to find and load the module. This protocol consists of two conceptual objects, [finders](https://docs.python.org/3/glossary.html#term-finder) and [loaders](https://docs.python.org/3/glossary.html#term-loader). A finder’s job is to determine whether it can find the named module using whatever strategy it knows about. Objects that implement both of these interfaces are referred to as [importers](https://docs.python.org/3/glossary.html#term-importer) - they return themselves when they find that they can load the requested module.

Python includes a number of default finders and importers. The first one knows how to locate built-in modules, and the second knows how to locate frozen modules. A third default finder searches an [import path](https://docs.python.org/3/glossary.html#term-import-path) for modules. The [import path](https://docs.python.org/3/glossary.html#term-import-path) is a list of locations that may name file system paths or zip files. It can also be extended to search for any locatable resource, such as those identified by URLs.

The import machinery is extensible, so new finders can be added to extend the range and scope of module searching.

Finders do not actually load modules. If they can find the named module, they return a *module spec*, an encapsulation of the module’s import-related information, which the import machinery then uses when loading the module.

The following sections describe the protocol for finders and loaders in more detail, including how you can create and register new ones to extend the import machinery.

*Changed in version 3.4:* In previous versions of Python, finders returned [loaders](https://docs.python.org/3/glossary.html#term-loader) directly, whereas now they return module specs which *contain* loaders. Loaders are still used during import but have fewer responsibilities.

> 如果在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中没有找到指定的模块，那么将调用 Python 的导入协议来寻找和加载模块。这个协议由两个概念性的对象组成，[finder](https://docs.python.org/3/glossary.html#term-finder) 和 [loaders](https://docs.python.org/3/glossary.html#term-loader) 。查找器（finders）的工作是确定它是否可以使用它所知道的任何策略找到指定的模块。实现这两个接口的对象被称为 [importers](https://docs.python.org/3/glossary.html#term-importer) - 当它们发现可以加载所请求的模块时就会返回自己。
>
> Python 包括一些默认的查找器和导入器。第一个知道如何定位内置模块，第二个知道如何定位冻结模块。第三个默认查找器在 [import path](https://docs.python.org/3/glossary.html#term-import-path) 中搜索模块。[import path](https://docs.python.org/3/glossary.html#term-import-path)是一个位置列表，可以命名为文件系统路径或压缩文件。它也可以被扩展为搜索任何可定位的资源，例如那些由URL识别的资源。
>
> 导入机制是可扩展的，因此可以添加新的搜索器来扩展模块搜索的范围。
>
> 搜索器实际上并不加载模块。如果他们能找到指定的模块，他们会返回一个*模块规格*，这是一个模块的导入相关信息的封装，然后导入机制在加载模块时使用它。
>
> 下面几节更详细地描述了查找器和装载器（loaders）的协议，包括你如何创建和注册新的查找器来扩展导入机制。
>
> *3.4版中的变化：* 在以前的Python版本中，查找器直接返回 [loaders](https://docs.python.org/3/glossary.html#term-loader)，而现在它们返回*包含*loaders的模块规格。在导入过程中仍然使用加载器，但它的职责更少。

### 5.3.3. Import hooks

The import machinery is designed to be extensible; the primary mechanism for this are the *import hooks*. There are two types of import hooks: *meta hooks* and *import path hooks*.

Meta hooks are called at the start of import processing, before any other import processing has occurred, other than [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) cache look up. This allows meta hooks to override [`sys.path`](https://docs.python.org/3/library/sys.html#sys.path) processing, frozen modules, or even built-in modules. Meta hooks are registered by adding new finder objects to [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path), as described below.

Import path hooks are called as part of [`sys.path`](https://docs.python.org/3/library/sys.html#sys.path) (or `package.__path__`) processing, at the point where their associated path item is encountered. Import path hooks are registered by adding new callables to [`sys.path_hooks`](https://docs.python.org/3/library/sys.html#sys.path_hooks) as described below.

> 导入机制被设计为可扩展的；其主要机制是*导入钩子*。有两种类型的导入钩子。*meta钩子*和*导入路径钩子*。
>
> meta钩子在导入处理开始时被调用，在任何其他导入处理发生之前，除了 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 缓存查询之外。这允许meta钩子覆盖 [`sys.path`](https://docs.python.org/3/library/sys.html#sys.path) 处理，冻结模块，甚至内置模块。meta钩子是通过向[`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path)添加新的查找对象来注册的，如下所述。
>
> 导入路径钩子作为 [`sys.path`](https://docs.python.org/3/library/sys.html#sys.path) (或 `package.__path__`) 处理的一部分被调用，在遇到其相关的路径项时。如下所述，通过向 [`sys.path_hooks`](https://docs.python.org/3/library/sys.html#sys.path_hooks) 添加新的可调用项来注册导入路径钩子。

### 5.3.4. The meta path

When the named module is not found in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), Python next searches [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path), which contains a list of meta path finder objects. These finders are queried in order to see if they know how to handle the named module. Meta path finders must implement a method called [`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec) which takes three arguments: a name, an import path, and (optionally) a target module. The meta path finder can use any strategy it wants to determine whether it can handle the named module or not.

If the meta path finder knows how to handle the named module, it returns a spec object. If it cannot handle the named module, it returns `None`. If [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path) processing reaches the end of its list without returning a spec, then a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) is raised. Any other exceptions raised are simply propagated up, aborting the import process.

The [`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec) method of meta path finders is called with two or three arguments. The first is the fully qualified name of the module being imported, for example `foo.bar.baz`. The second argument is the path entries to use for the module search. For top-level modules, the second argument is `None`, but for submodules or subpackages, the second argument is the value of the parent package’s `__path__` attribute. If the appropriate `__path__` attribute cannot be accessed, a [`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError) is raised. The third argument is an existing module object that will be the target of loading later. The import system passes in a target module only during reload.

The meta path may be traversed multiple times for a single import request. For example, assuming none of the modules involved has already been cached, importing `foo.bar.baz` will first perform a top level import, calling `mpf.find_spec("foo", None, None)` on each meta path finder (`mpf`). After `foo` has been imported, `foo.bar` will be imported by traversing the meta path a second time, calling `mpf.find_spec("foo.bar", foo.__path__, None)`. Once `foo.bar` has been imported, the final traversal will call `mpf.find_spec("foo.bar.baz", foo.bar.__path__, None)`.

Some meta path finders only support top level imports. These importers will always return `None` when anything other than `None` is passed as the second argument.

Python’s default [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path) has three meta path finders, one that knows how to import built-in modules, one that knows how to import frozen modules, and one that knows how to import modules from an [import path](https://docs.python.org/3/glossary.html#term-import-path) (i.e. the [path based finder](https://docs.python.org/3/glossary.html#term-path-based-finder)).

*Changed in version 3.4:* The [`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec) method of meta path finders replaced [`find_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_module), which is now deprecated. While it will continue to work without change, the import machinery will try it only if the finder does not implement `find_spec()`.

*Changed in version 3.10:* Use of [`find_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_module) by the import system now raises [`ImportWarning`](https://docs.python.org/3/library/exceptions.html#ImportWarning).

> 如果在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中没有找到指定的模块，Python 接下来会搜索 [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path)，它包含一个元路径查找器对象的列表。这些查找器被查询，以查看它们是否知道如何处理指定的模块。元路径查找器必须实现一个名为[`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec)的方法，该方法接受三个参数：名称、导入路径和（可选）目标模块。元路径查找器可以使用任何它想要的策略来确定它是否可以处理命名的模块。
>
> 如果元路径查找器知道如何处理命名的模块，它将返回一个规格对象。如果它不能处理命名的模块，它将返回 `None`。如果[`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path)处理到其列表的末尾而没有返回一个spec，那么会引发[`moduleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError)。任何其他的异常都会向上传播，中止导入过程。
>
> 元路径搜索器的[`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec)方法被调用，有两个或三个参数。第一个参数是被导入模块的全称，例如`foo.bar.baz`。第二个参数是用于模块搜索的路径条目。对于顶级模块，第二个参数是`None`，但是对于子模块或子包，第二个参数是父包的 `__path__` 属性的值。如果适当的 `__path__` 属性不能被访问，会产生一个[`ModuleNotFoundError`](https://docs.python.org/3/library/exceptions.html#ModuleNotFoundError)。第三个参数是一个现有的模块对象，它将成为以后加载的目标。导入系统只在重载时传入一个目标模块。
>
> 对于一个导入请求，元路径可能会被多次遍历。例如，假设所涉及的模块都没有被缓存，导入`foo.bar.baz`将首先执行顶级导入，在每个元路径查找器（`mpf`）上调用`mpf.find_spec("foo", None, None)`。在`foo`被导入后，`foo.bar`将通过第二次遍历元路径被导入，调用`mpf.find_spec("foo.bar", foo.__path__, None)`。一旦`foo.bar`被导入，最后的遍历将调用`mpf.find_spec("foo.bar.baz", foo.bar.__path__, None)`。
>
> 一些元路径查找器只支持顶层导入。当第二个参数是 `None` 以外的东西时，这些导入器将总是返回 `None`。
>
> Python 默认的 [`sys.meta_path`](https://docs.python.org/3/library/sys.html#sys.meta_path) 有三个元路径查找器，一个知道如何导入内置模块，一个知道如何导入冻结模块，还有一个知道如何从 [import path](https://docs.python.org/3/glossary.html#term-import-path) 导入模块 (即 [path based finder](https://docs.python.org/3/glossary.html#term-path-based-finder)) 。
>
> *在3.4版本中有所改变：*元路径查找器的[`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec)方法取代了[`find_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_module)，后者现在已经被废弃。虽然它将继续工作，但只有在查找器没有实现`find_spec()`时，导入机制才会尝试它。
>
> *在3.10版本中改变了：*导入系统使用[`find_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_module)现在会引发[`ImportWarning`](https://docs.python.org/3/library/exceptions.html#ImportWarning)。

## 5.4. Loading

If and when a module spec is found, the import machinery will use it (and the loader it contains) when loading the module. Here is an approximation of what happens during the loading portion of import:

> 如果找到了模块规范，导入机制将在加载模块时使用它（以及它所包含的加载器）。下面是导入过程中加载部分发生的近似情况：

Note the following details:

> - If there is an existing module object with the given name in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), import will have already returned it.
> - The module will exist in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) before the loader executes the module code. This is crucial because the module code may (directly or indirectly) import itself; adding it to [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) beforehand prevents unbounded recursion in the worst case and multiple loading in the best.
> - If loading fails, the failing module – and only the failing module – gets removed from [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules). Any module already in the [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) cache, and any module that was successfully loaded as a side-effect, must remain in the cache. This contrasts with reloading where even the failing module is left in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules).
> - After the module is created but before execution, the import machinery sets the import-related module attributes (“_init_module_attrs” in the pseudo-code example above), as summarized in a [later section](https://docs.python.org/3/reference/import.html#import-mod-attrs).
> - Module execution is the key moment of loading in which the module’s namespace gets populated. Execution is entirely delegated to the loader, which gets to decide what gets populated and how.
> - The module created during loading and passed to exec_module() may not be the one returned at the end of import [2](https://docs.python.org/3/reference/import.html#fnlo).

*Changed in version 3.4:* The import system has taken over the boilerplate responsibilities of loaders. These were previously performed by the [`importlib.abc.Loader.load_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.load_module) method.

> 注意以下细节：
>
> > - 如果在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中有一个给定名称的现有模块对象，import将已经返回。
> > - 在加载器执行模块代码之前，模块将存在于 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中。这很关键，因为模块代码可能（直接或间接）导入自己；事先将其添加到 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中，在最坏的情况下防止无限的递归，在最好的情况下防止多次加载。
> > - 如果加载失败，失败的模块--只有失败的模块--会从 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中删除。任何已经在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 缓存中的模块，以及任何作为副作用成功加载的模块，必须保留在缓存中。这与重新加载形成对比，即使是失败的模块也被留在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中。
> > - 在模块创建之后但在执行之前，导入机制设置与导入有关的模块属性（上面的伪代码例子中的"_init_module_attrs"），正如[后面的章节](https://docs.python.org/3/reference/import.html#import-mod-attrs)中所总结的。
> > - 模块的执行是加载的关键时刻，在这个时刻，模块的名字空间被填充。执行是完全委托给加载器的，它可以决定什么被填充以及如何填充。
> > - 在加载过程中创建并传递给exec_module()的模块可能不是导入结束时返回的模块[^2]。
>
> *3.4版本中的变化：*导入系统已经接管了加载器的模板责任。这些以前是由[`importlib.abc.Loader.load_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.load_module)方法执行的。

### 5.4.1. Loaders

Module loaders provide the critical function of loading: module execution. The import machinery calls the [`importlib.abc.Loader.exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module) method with a single argument, the module object to execute. Any value returned from [`exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module) is ignored.

Loaders must satisfy the following requirements:

> - If the module is a Python module (as opposed to a built-in module or a dynamically loaded extension), the loader should execute the module’s code in the module’s global name space (`module.__dict__`).
> - If the loader cannot execute the module, it should raise an [`ImportError`](https://docs.python.org/3/library/exceptions.html#ImportError), although any other exception raised during [`exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module) will be propagated.

In many cases, the finder and loader can be the same object; in such cases the [`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec) method would just return a spec with the loader set to `self`.

Module loaders may opt in to creating the module object during loading by implementing a [`create_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.create_module) method. It takes one argument, the module spec, and returns the new module object to use during loading. `create_module()` does not need to set any attributes on the module object. If the method returns `None`, the import machinery will create the new module itself.

*New in version 3.4:* The [`create_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.create_module) method of loaders.

*Changed in version 3.4:* The [`load_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.load_module) method was replaced by [`exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module) and the import machinery assumed all the boilerplate responsibilities of loading.

For compatibility with existing loaders, the import machinery will use the `load_module()` method of loaders if it exists and the loader does not also implement `exec_module()`. However, `load_module()` has been deprecated and loaders should implement `exec_module()` instead.

The `load_module()` method must implement all the boilerplate loading functionality described above in addition to executing the module. All the same constraints apply, with some additional clarification:

> - If there is an existing module object with the given name in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), the loader must use that existing module. (Otherwise, [`importlib.reload()`](https://docs.python.org/3/library/importlib.html#importlib.reload) will not work correctly.) If the named module does not exist in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), the loader must create a new module object and add it to [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules).
> - The module *must* exist in [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) before the loader executes the module code, to prevent unbounded recursion or multiple loading.
> - If loading fails, the loader must remove any modules it has inserted into [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules), but it must remove **only** the failing module(s), and only if the loader itself has loaded the module(s) explicitly.

*Changed in version 3.5:* A [`DeprecationWarning`](https://docs.python.org/3/library/exceptions.html#DeprecationWarning) is raised when `exec_module()` is defined but `create_module()` is not.

*Changed in version 3.6:* An [`ImportError`](https://docs.python.org/3/library/exceptions.html#ImportError) is raised when `exec_module()` is defined but `create_module()` is not.

*Changed in version 3.10:* Use of `load_module()` will raise [`ImportWarning`](https://docs.python.org/3/library/exceptions.html#ImportWarning).

> 模块加载器提供加载的关键功能：模块执行。导入机制调用 [`importlib.abc.Loader.exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module) 方法，该方法只有一个参数，即要执行的模块对象。任何从[`exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module)返回的值都被忽略了。
>
> 加载器必须满足以下要求：
>
> > - 如果模块是一个 Python 模块 (相对于内置模块或动态加载的扩展)，加载器应该在模块的全局名称空间 (`module.__dict__`) 执行模块的代码。
> > - 如果加载器不能执行该模块，它应该引发一个[`ImportError`](https://docs.python.org/3/library/exceptions.html#ImportError)，尽管在[`exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module)期间引发的任何其他异常都会被传播。
>
> 在许多情况下，查找器和加载器可以是同一个对象；在这种情况下，[`find_spec()`](https://docs.python.org/3/library/importlib.html#importlib.abc.MetaPathFinder.find_spec)方法将只是返回一个规格，加载器设置为`self`。
>
> 模块加载器可以通过实现 [`create_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.create_module) 方法来选择在加载时创建模块对象。它需要一个参数，即模块规格，并返回新的模块对象，以便在加载过程中使用。`create_module()`不需要在模块对象上设置任何属性。如果该方法返回 `None`，导入机制将自己创建新模块。
>
> *在3.4版本中新增：*加载器的[`create_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.create_module)方法。
>
> *在3.4版本中改变了：* [`load_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.load_module)方法被[`exec_module()`](https://docs.python.org/3/library/importlib.html#importlib.abc.Loader.exec_module)取代，导入机制承担了所有加载的模板责任。
>
> 为了与现有的加载器兼容，如果加载器存在 `load_module()`方法，并且加载器没有实现`exec_module()`，导入机制将使用该方法。然而，`load_module()`已经被弃用，加载器应该实现`exec_module()`来代替。
>
> `load_module()`方法除了执行模块外，还必须实现上面描述的所有模板加载功能。所有相同的限制条件都适用，还有一些额外的说明：
>
> > - 如果在 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中有一个给定名称的现有模块对象，加载器必须使用该现有模块。(否则，[`importlib.reload()`](https://docs.python.org/3/library/importlib.html#importlib.reload)将不能正常工作）。如果命名的模块不存在于 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中，加载器必须创建一个新的模块对象并将其添加到 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中。
> > - 在加载器执行模块代码之前，模块*必须*存在于 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中，以防止无限制的递归或多重加载。
> > - 如果加载失败，加载器必须删除它插入到 [`sys.modules`](https://docs.python.org/3/library/sys.html#sys.modules) 中的任何模块，但它必须**只**删除失败的模块，而且只有在加载器本身明确加载了该模块的情况下。
>
> *在3.5版中改变了：* 当 `exec_module()` 被定义而 `create_module()` 没有被定义时，会发出一个[`DeprecationWarning`](https://docs.python.org/3/library/exceptions.html#DeprecationWarning)。
>
> *在3.6版中有所改变：* 当 `exec_module()` 被定义但`create_module()`没有被定义时，会产生一个[`ImportError`](https://docs.python.org/3/library/exceptions.html#ImportError)。
>
> *在3.10版本中改变了：*使用 `load_module()` 将引发[`ImportWarning`](https://docs.python.org/3/library/exceptions.html#ImportWarning)。

### 5.4.2. Submodules

When a submodule is loaded using any mechanism (e.g. `importlib` APIs, the `import` or `import-from` statements, or built-in `__import__()`) a binding is placed in the parent module’s namespace to the submodule object. For example, if package `spam` has a submodule `foo`, after importing `spam.foo`, `spam` will have an attribute `foo` which is bound to the submodule. Let’s say you have the following directory structure:

> 当一个子模块使用任何机制（例如`importlib`API，`import`或`import-from`语句，或内置的`__import__()`）加载时，在父模块的名称空间中会有一个绑定到子模块对象。例如，如果包 `spam` 有一个子模块 `foo`，在导入 `spam.foo` 后，`spam`将有一个属性 `foo` 被绑定到子模块。假设你有以下的目录结构：

and `spam/__init__.py` has the following line in it:

> 而 `spam/__init__.py` 中有以下一行：

then executing the following puts name bindings for `foo` and `Foo` in the `spam` module:

> 然后执行下面的命令，将 `foo` 和 `Foo` 的名字绑定到 `spam` 模块：

Given Python’s familiar name binding rules this might seem surprising, but it’s actually a fundamental feature of the import system. The invariant holding is that if you have `sys.modules['spam']` and `sys.modules['spam.foo']` (as you would after the above import), the latter must appear as the `foo` attribute of the former.

> 考虑到 Python 熟悉的名字绑定规则，这似乎令人惊讶，但这实际上是导入系统的一个基本特征。不变的是，如果你有 `sys.modules['spam']` 和 `sys.modules['spam.foo']` (就像上面的导入后一样)，后者必须作为前者的 `foo` 属性出现。

### 5.4.3. Module spec

The import machinery uses a variety of information about each module during import, especially before loading. Most of the information is common to all modules. The purpose of a module’s spec is to encapsulate this import-related information on a per-module basis.

Using a spec during import allows state to be transferred between import system components, e.g. between the finder that creates the module spec and the loader that executes it. Most importantly, it allows the import machinery to perform the boilerplate operations of loading, whereas without a module spec the loader had that responsibility.

The module’s spec is exposed as the `__spec__` attribute on a module object. See [`ModuleSpec`](https://docs.python.org/3/library/importlib.html#importlib.machinery.ModuleSpec) for details on the contents of the module spec.

*New in version 3.4.*

> 在导入过程中，尤其是在加载之前，导入机制会使用关于每个模块的各种信息。大多数信息对所有模块都是通用的。一个模块的规范的目的是在每个模块的基础上封装这些与导入有关的信息。
>
> 在导入过程中使用规范允许状态在导入系统组件之间转移，例如在创建模块规范的查找器和执行它的加载器之间。最重要的是，它允许导入机制执行加载的模板操作，而在没有模块规范的情况下，加载器有这个责任。
>
> 模块的规格以模块对象上的 `__spec__` 属性的形式公开。参见 [`ModuleSpec`](https://docs.python.org/3/library/importlib.html#importlib.machinery.ModuleSpec) 了解有关模块规格内容的细节。
>
> *在3.4 版本中新增*

### 5.4.4. Import-related module attributes